<a href="https://colab.research.google.com/github/Lu-David/CovariateShift/blob/main/Covariate_Shift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python implementation of 2014 Liu Paper

https://proceedings.neurips.cc/paper/2014/file/d67d8ab4f4c10bf22aa353e27879133c-Paper.pdf

### Summary
Problem: sample selection bias

## Binary Classification

In [1]:
import scipy.io
import numpy as np
import os

folder_path = '/content' # TODO Upload data files and / or specify path

x_1 = scipy.io.loadmat(os.path.join(folder_path, 'x_1.mat'))['x_1']
x_2 = scipy.io.loadmat(os.path.join(folder_path, 'x_2.mat'))['x_2']
y_1 = np.transpose(scipy.io.loadmat(os.path.join(folder_path, 'y_1.mat'))['y_1'])
y_2 = np.transpose(scipy.io.loadmat(os.path.join(folder_path, 'y_2.mat'))['y_2'])

In [ ]:
mu_s = [6, 6] # source mean 
var_s = [[3, -2], [-2, 3]] # source variance
mu_t = [7, 7] # target mean
var_t = [[3, 2], [2, 3]] # target variance

```
Our approach, in contrast, obtains an estimate for
each given target distribution that is robust to all the conditional label distributions matching source
statistics. While having an exact or well-estimated target distribution a priori may not be possible
for some applications, large amounts of unlabeled data enable this in many batch learning settings.
```

**Question:** How would you determine the source / target mean / variance in a real world setting? The values that I've set above don't seem to match the values loaded from the matlab data objects. 


### Training

In [ ]:
#
#
#
# r_ts : ratio of target / source density distribution for importance weighting method 
def binaryRobustTrain(X_s, y_s, r_st, r_ts, lamb = 0.001, lr = 1, max_itr = 10000, min_gradient = 0.0001):
  n_row, n_col = X_s.shape

  F = np.concatenate((np.ones((n_row, 1)), X_s), axis=1) 
  F_g = F * np.tile(r_ts, n_col + 1)

  P = np.zeros((n_row, 1)) 
  S_g = np.ones((n_col + 1, 1)) * 1e-8

  t = 1
  theta = np.ones((n_col + 1, 1)) 
  l_0 = 0
  l_1 = (1 + (1 + (4 * l_0 ** 2)) ** 0.5) / 2
  delta_1 = 0

  while True:
    t = t + 1
    decay = np.sqrt(1000 / (1000 + t))

    l_2 = (1 + (1 + (4 * l_1 ** 2)) ** 0.5) / 2
    l_3 = (1 - l_1) / l_2

    # TODO: Vectorize? 
    for i in range(n_row):
      W = r_st[i]
      temp =  (np.dot(np.transpose(theta), np.transpose(F[i, :])) * y_s[i] * W)[0]      
      temp_max = max(temp, -1 * temp)  # multiplied by negative one because Y is -1, 1 instead of 0, 1
      temp_min = min(temp, -1 * temp)
      P[i] = np.exp(temp - temp_max - np.log(1 + np.exp(temp_min - temp_max))) # Question: np.exp(temp) refers to Theorem 2, but what does temp_max - np.log(everything else) do? Is this some sort of correction term? 

    # G : Gradient which is the logloss gradient calculated as P 
    G = np.transpose(np.dot(np.transpose(P * y_s), F_g)) - np.transpose(np.dot(np.transpose(y_s), F_g)) + 2 * lamb * theta
    if np.linalg.norm(G) < min_gradient:
      print('Optimization stops by reaching minimum gradient.')
      break

    S_g = S_g + G ** 2
    delta_2 = theta - np.dot(decay, lr) * G / np.sqrt(S_g) 
    theta = np.dot((1 - l_3), delta_2) + np.dot(l_3, delta_1)
    delta_1 = delta_2
    l_1 = l_2

    if t > max_itr:
      print("Optimizination stops by reaching maximum iteration")
      break
  return theta

In [ ]:
# Robust learning

from scipy.stats import multivariate_normal

mvn_s = multivariate_normal(mu_s, var_s)
mvn_t = multivariate_normal(mu_t, var_t)

d_s = mvn_s.pdf(x_1)
d_t = mvn_t.pdf(x_1)

In [ ]:
# Robust Learning
theta_1 = binaryRobustTrain(x_1, y_1, d_s / d_t, np.ones((x_1.shape[0], 1)))
print("Weights", theta_1)

Optimizination stops by reaching maximum iteration
Weights [[11.91705313]
 [-1.00605443]
 [-1.0069282 ]]


In [ ]:
# Logistic Regression
theta_2 = binaryRobustTrain(x_1, y_1, np.ones((x_1.shape[0], 1)), np.ones((x_1.shape[0], 1)))
print("Weights", theta_2)

Optimization stops by reaching minimum gradient.
Weights [[12.35379291]
 [-1.04099695]
 [-1.0359673 ]]


In [ ]:
# Importance Weighting
r_ts = d_t / d_s
theta_3 = binaryRobustTrain(x_1, y_1, np.ones((x_1.shape[0], 1)), r_ts.reshape(r_ts.shape[0], 1))
print("Weights", theta_3)

Optimization stops by reaching minimum gradient.
Weights [[ 9.40884544]
 [-0.76023811]
 [-0.85193742]]


### Testing

In [ ]:
def binaryRobustTest(theta, X_t, y_t, r_st):
  n_row, _ = X_t.shape

  # F : Features
  F = np.concatenate((np.ones((n_row, 1)), X_t), axis=1) 

  # P : predictions 
  P = np.zeros((n_row, 1))
  logloss = 0
  prediction = np.zeros((n_row, 2))
  for i in range(n_row):
    W = r_st[i]
    temp =  (np.dot(np.transpose(theta), np.transpose(F[i, :])) * y_t[i] * W)[0]      
    temp_max = max(temp, -1 * temp)
    temp_min = min(temp, -1 * temp)
    P[i] = np.exp(temp - temp_max - np.log(1 + np.exp(temp_min - temp_max)))
    logloss = logloss - np.log(P[i])

    if y_t[i] == 1:
      prediction[i] = [P[i], 1 - P[i]]
    else:
      prediction[i] = [1 - P[i], P[i]]

  logloss = logloss / n_row / 0.6931
  return logloss, prediction

def computeAcc(pred, y):
  n_row, n_class = pred.shape
  
  max_ind = np.argmax(pred, axis = 1)

  summ = 0

  if n_class == 2:
    for i in range(n_row):
      if max_ind[i] == 1 and y[i] == -1:
        summ += 1
      elif max_ind[i] == 0 and y[i] == 1:
        summ += 1
  else:
    summ = sum(np.argmax(pred, axis = 1) == y_t - 1)

  return summ / n_row

In [ ]:
# Robost 
d_s = mvn_s.pdf(x_2)
d_t = mvn_t.pdf(x_2)

In [ ]:
logloss_1, pred_1 = binaryRobustTest(theta_1, x_2, y_2, d_s / d_t)

[0.82562143] [[4.98721790e-01 5.01278210e-01]
 [3.75410707e-01 6.24589293e-01]
 [2.87288358e-01 7.12711642e-01]
 [4.92253351e-01 5.07746649e-01]
 [9.99926840e-01 7.31602132e-05]
 [2.58896032e-01 7.41103968e-01]
 [6.92374255e-01 3.07625745e-01]
 [1.19306263e-01 8.80693737e-01]
 [4.80252940e-01 5.19747060e-01]
 [9.14501821e-01 8.54981788e-02]
 [9.37599379e-01 6.24006210e-02]
 [5.63100285e-01 4.36899715e-01]
 [3.77913750e-01 6.22086250e-01]
 [4.44454276e-01 5.55545724e-01]
 [3.77350230e-02 9.62264977e-01]
 [5.35448942e-02 9.46455106e-01]
 [1.07016685e-01 8.92983315e-01]
 [4.99622400e-01 5.00377600e-01]
 [1.25002038e-01 8.74997962e-01]
 [1.42800777e-01 8.57199223e-01]
 [3.78958754e-01 6.21041246e-01]
 [4.30090664e-01 5.69909336e-01]
 [4.99999675e-01 5.00000325e-01]
 [4.16871304e-01 5.83128696e-01]
 [4.56561259e-01 5.43438741e-01]
 [3.39346405e-01 6.60653595e-01]
 [4.98707133e-01 5.01292867e-01]
 [3.58463613e-01 6.41536387e-01]
 [4.99982035e-01 5.00017965e-01]
 [1.93031286e-01 8.06968714e-0

In [ ]:
logloss_2, pred_2 = binaryRobustTest(theta_2, x_2, y_2, np.ones((x_1.shape[0], 1)))
print(logloss_2)

[0.78499119]


In [ ]:
logloss_3, pred_3 = binaryRobustTest(theta_3, x_2, y_2, np.ones((x_1.shape[0], 1)))
print(logloss_3)

[0.69593336]


In [ ]:
# What do these results mean? 
print(computeAcc(pred_1, y_2))
print(computeAcc(pred_2, y_2))
print(computeAcc(pred_3, y_2))

0.89
0.9
0.87


## MultiClass Classification

In [ ]:
import scipy.io
import numpy as np
import os

folder_path = '/content' # TODO Change this

iris_train = scipy.io.loadmat(os.path.join(folder_path, 'iris_train.mat'))['iris_train']
iris_test = scipy.io.loadmat(os.path.join(folder_path, 'iris_test.mat'))['iris_test']

X_s = iris_train[:,0:-1]
y_s = iris_train[:, -1]
X_t = iris_test[:,0:-1]
y_t = iris_test[:,-1]


### Training


In [ ]:
def LRDensityEstimation(X_s, X_t, lambdas = [0.0625, 1, 16]):

  np.random.seed(10) # seed set in matlab code as well

  ns_row, _ = X_s.shape
  nt_row, _ = X_t.shape

  inda_s = np.arange(ns_row)
  inda_t = np.arange(nt_row)

  nv_s = int(np.floor(0.2 * ns_row))
  nv_t = int(np.floor(0.2 * nt_row))

  indv_s = np.array([92, 3, 74, 87, 57, 26, 23, 85, 19, 10, 75, 103, 1, 54, 111, 64, 116, 30, 118, 71, 105, 14, 36]) - 1 # indv_s = np.random.permutation(ns_row)[:nv_s] # 

  indv_t = np.array([22, 14, 31, 18, 15, 29]) - 1 # np.random.permutation(nt_row)[:nv_t]

  indt_s = np.setdiff1d(inda_s, indv_s)
  
  indt_t = np.setdiff1d(inda_t, indv_t)

  X_train = np.concatenate((X_s[indt_s, :], X_t[indt_t, :]))
  X_valid = np.concatenate((X_s[indv_s, :], X_t[indv_t, :]))
  
  y_train = np.concatenate((np.ones((ns_row - nv_s, 1)), -1 * np.ones((nt_row - nv_t, 1)) ))
  y_valid = np.concatenate((np.ones((nv_s, 1)), -1 * np.ones((nv_t, 1)) ))

  rt_st = np.ones((ns_row + nt_row - nv_s - nv_t, 1))
  rv_st = np.ones((nv_s + nv_t, 1))
  
  logloss = np.zeros((len(lambdas), 1))
  for i, lamb in enumerate(lambdas):
    theta = binaryRobustTrain(X_train, y_train, rt_st, rt_st, lamb=lamb,min_gradient=0.1)
    _, pred = binaryRobustTest(theta, X_valid, y_valid, rv_st )
    logloss[i] = (-sum(np.log(pred[:nv_s, 0])) - sum(np.log(pred[nv_s: nv_s + nv_t, 1]))) / (nv_s + nv_t) / 0.6931

  ind_min = np.argmin(logloss)

  X_train = np.concatenate((X_s, X_t))
  y_train = np.concatenate((np.ones((ns_row, 1)), -1 * np.ones((nt_row, 1)) ))
  r_st = np.ones((ns_row + nt_row, 1))

  theta = binaryRobustTrain(X_train, y_train, r_st, r_st, lambdas[ind_min])
  _, pred = binaryRobustTest(theta, X_train, y_train, r_st)
  print(pred)

  d_ss = pred[:ns_row, 0]
  d_st = pred[:ns_row, 1]

  d_ts = pred[ns_row:, 0]
  d_tt = pred[ns_row:, 1]

  return d_ss, d_st, d_ts, d_tt

d_ss, d_st, d_ts, d_tt = LRDensityEstimation(X_s, X_t, [0.1, 1, 10])

Optimization stops by reaching minimum gradient.
Optimization stops by reaching minimum gradient.
Optimization stops by reaching minimum gradient.
Optimization stops by reaching minimum gradient.
[[4.89220620e-01 5.10779380e-01]
 [7.71536817e-01 2.28463183e-01]
 [6.45858005e-01 3.54141995e-01]
 [7.95041119e-01 2.04958881e-01]
 [8.88689504e-01 1.11310496e-01]
 [7.48191866e-01 2.51808134e-01]
 [5.32154209e-01 4.67845791e-01]
 [7.93256883e-01 2.06743117e-01]
 [6.16145836e-01 3.83854164e-01]
 [5.87216006e-01 4.12783994e-01]
 [6.32044796e-01 3.67955204e-01]
 [5.89128479e-01 4.10871521e-01]
 [5.30824243e-01 4.69175757e-01]
 [4.96827171e-01 5.03172829e-01]
 [6.53010165e-01 3.46989835e-01]
 [6.40233768e-01 3.59766232e-01]
 [7.72793798e-01 2.27206202e-01]
 [6.13764757e-01 3.86235243e-01]
 [4.71216716e-01 5.28783284e-01]
 [6.84203432e-01 3.15796568e-01]
 [5.82563687e-01 4.17436313e-01]
 [6.14509614e-01 3.85490386e-01]
 [9.95133571e-01 4.86642948e-03]
 [9.90252858e-01 9.74714240e-03]
 [9.95120057

In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

def multiClassRobustTrain(X_s, y_s, n_class, r_st, r_ts, lamb = 0.1, lr = 0.01, max_itr = 100000, min_gradient = 0.001):
  n_row, n_col = X_s.shape

  # lamb : regularization matrix (3 x 4) 
  lamb = lamb.reshape(-1, 1)  
  lamb = np.transpose(np.tile(lamb, n_class))
  
  # F : features 
  F = X_s 

  # F_g : features reweighted to match source / target distribution 
  F_g = F * np.tile(r_ts, n_col)

  # Y : binarized matrix representing labels
  Y = lb.fit_transform(y_s)

  # P : predictions 
  P = np.zeros((n_row, n_class)) 

  # S_g : 
  S_g = np.ones((n_class, n_col)) * 1e-8

  t = 1

  # theta : weights matrix
  theta = np.ones((n_class, n_col)) # not randomly assigned? 

  # l_0 : ______ 
  l_0 = 0

  # l_1 : 
  l_1 = (1 + (1 + (4 * l_0 ** 2)) ** 0.5) / 2
  delta_1 = 0

  while True:
    t = t + 1
    decay = np.sqrt(1000 / (1000 + t))
    l_2 = (1 + (1 + (4 * l_1 ** 2)) ** 0.5) / 2
    l_3 = (1 - l_1) / l_2

    for i in range(n_row):
      W = r_st[i]
      temp =  np.dot(theta, np.transpose(F[i, :])) * W
      temp = temp - np.max(temp)
      sum_temp = sum(np.exp(temp))
      P[i] = np.exp(temp - np.log(sum_temp))

    G = np.dot(np.transpose(P) - np.transpose(Y), F_g) + 2 * lamb * theta

    if np.linalg.norm(G) < min_gradient:
      print('Optimization stops by reaching minimum gradient.')
      break
    elif t > max_itr:
      print("Optimizination stops by reaching maximum iteration")
      break

    S_g = S_g + G ** 2
    delta_2 = theta - decay * lr * G / np.sqrt(S_g) 
    theta = np.dot((1 - l_3), delta_2) + np.dot(l_3, delta_1)
    delta_1 = delta_2
    l_1 = l_2
    
  return theta

This density estimation algorithm isn't described in the paper right? \
Question: No bias? \
Question: What is d_ss and d_st? Isn't d_ss just 1 - d_st? 


In [ ]:
# Robust learning

ns_row, n_col = X_s.shape

n_class = int(max(y_s))

lamb = 2 * np.std(X_s, axis=0, ddof=1) / np.sqrt(ns_row)
lamb[0] = 1

theta_robust = multiClassRobustTrain(X_s, y_s, n_class, d_ss / d_st, np.ones((ns_row, 1)), lamb=lamb, lr = 1, min_gradient=0.01)

Optimization stops by reaching minimum gradient.


In [ ]:
theta_lr = multiClassRobustTrain(X_s, y_s, n_class, np.ones((ns_row, 1)), np.ones((ns_row, 1)), lamb=lamb, lr = 1, min_gradient=0.01)

Optimization stops by reaching minimum gradient.


In [ ]:
r_ts = np.reshape(d_st / d_ss, (-1, 1))
theta_iw = multiClassRobustTrain(X_s, y_s, n_class, np.ones((ns_row, 1)), r_ts, lamb=lamb, lr = 1, min_gradient=0.01)

Optimization stops by reaching minimum gradient.


### Testing

In [ ]:
def multiClassRobustTest(theta, X_t, y_t, n_class, r_st):
  n_row, _ = X_t.shape
  F = X_t 
  P = np.zeros((n_row, n_class))
  logloss = 0 

  for i in range(n_row):
      W = r_st[i]
      temp =  np.dot(theta, np.transpose(F[i, :])) * W
      temp = temp - np.max(temp)
      sum_temp = sum(np.exp(temp))
      P[i] = np.exp(temp - np.log(sum_temp))
      logloss -= np.log(P[i, int(y_t[i]) - 1])

  logloss = logloss / n_row / 0.6931
  return logloss, P

In [ ]:
logloss, pred = multiClassRobustTest(theta_robust, X_t, y_t, n_class, d_ts / d_tt)
acc = computeAcc(pred, y_t)
print(f'Acc is {acc} and logloss is {logloss} for robust method')

Acc is 1.0 and logloss is 0.4989368812192846 for robust method


In [ ]:
logloss, pred = multiClassRobustTest(theta_lr, X_t, y_t, n_class, np.ones((ns_row, 1)))
acc = computeAcc(pred, y_t)
print(f'Acc is {acc} and logloss is {logloss} for LR method')

Acc is 1.0 and logloss is 0.09988054970933052 for IR method


In [ ]:
logloss, pred = multiClassRobustTest(theta_iw, X_t, y_t, n_class, np.ones((ns_row, 1)))
acc = computeAcc(pred, y_t)
print(f'Acc is {acc} and logloss is {logloss} for IW method')

Acc is 0.96875 and logloss is 0.17910794400844812 for IW method


Isn't there a data leakage problem? 
Thinking that in a healthcare setting, you may not know the target distribution as data is coming in one after another. 
